In [5]:
import pandas as pd
import os

/Users/alessandro/Desktop/ML/progetto


In [29]:
# ransomware dataset
ransomware_file_dir = os.path.join(os.getcwd(), 'dataset', 'Ransomware', 'WannaLocker')

ransomware_fils = os.listdir(ransomware_file_dir)

ransomware_dfs = [pd.read_csv(os.path.join(ransomware_file_dir, f)) for f in ransomware_fils]
ransomware_df = pd.concat(ransomware_dfs, ignore_index=True)
print(len(ransomware_df))
ransomware_df.head()

32701


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,172.217.3.106-10.42.0.151-443-40915-6,10.42.0.151,40915,172.217.3.106,443,6,23/08/2017 12:02:34,37553,1,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RANSOMWARE_WANNALOCKER
1,10.42.0.151-31.13.80.5-56997-443-6,10.42.0.151,56997,31.13.80.5,443,6,23/08/2017 12:02:37,222014,6,7,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RANSOMWARE_WANNALOCKER
2,172.217.3.106-10.42.0.151-443-57436-6,10.42.0.151,57436,172.217.3.106,443,6,23/08/2017 12:02:38,116,2,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RANSOMWARE_WANNALOCKER
3,157.240.2.35-10.42.0.151-443-35237-6,10.42.0.151,35237,157.240.2.35,443,6,23/08/2017 12:02:54,109324,8,5,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RANSOMWARE_WANNALOCKER
4,157.240.2.35-10.42.0.151-443-35237-6,157.240.2.35,443,10.42.0.151,35237,6,23/08/2017 12:02:54,20069,5,2,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,RANSOMWARE_WANNALOCKER


In [36]:
# benign dataset
benign_file_dir = os.path.join(os.getcwd(), 'dataset', 'Benign', 'Benign2015')

benign_fils = os.listdir(benign_file_dir)
benign_dfs = [pd.read_csv(os.path.join(benign_file_dir, f), sep=',') for f in benign_fils[:55]]
benign_df = pd.concat(benign_dfs, ignore_index=True)


print(benign_df.shape)
benign_df

(34389, 85)


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,157.240.18.36-10.42.0.211-443-45748-6,157.240.18.36,443,10.42.0.211,45748,6,06/08/2017 09:13:30,54,2,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,157.240.18.36-10.42.0.211-443-45748-6,157.240.18.36,443,10.42.0.211,45748,6,06/08/2017 09:13:30,103975,1,4,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,172.217.6.234-10.42.0.211-443-39858-6,10.42.0.211,39858,172.217.6.234,443,6,06/08/2017 09:13:43,36851,1,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,172.217.6.234-10.42.0.211-443-45344-6,10.42.0.211,45344,172.217.6.234,443,6,06/08/2017 09:13:51,36893,1,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,172.217.6.193-10.42.0.211-443-34861-6,10.42.0.211,34861,172.217.6.193,443,6,06/08/2017 09:13:52,36780,1,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34384,10.42.0.211-10.42.0.1-10977-53-17,10.42.0.211,10977,10.42.0.1,53,17,04/08/2017 10:22:50,24524,1,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
34385,10.42.0.211-23.208.168.108-41402-443-6,10.42.0.211,41402,23.208.168.108,443,6,04/08/2017 10:29:01,479,2,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
34386,10.42.0.211-104.244.42.66-38103-443-6,10.42.0.211,38103,104.244.42.66,443,6,04/08/2017 10:22:54,15960957,9,7,...,32,215250.0,0.0,215250.0,215250.0,15745707.0,0.0,15745707.0,15745707.0,BENIGN
34387,199.59.148.73-10.42.0.211-443-46731-6,10.42.0.211,46731,199.59.148.73,443,6,04/08/2017 10:22:50,10344527,9,7,...,32,395259.0,0.0,395259.0,395259.0,9949268.0,0.0,9949268.0,9949268.0,BENIGN
